In [1]:
import pandas as pd
from pathlib import PurePath
import numpy as np
import datetime as dt
from pandas.tseries.holiday import USFederalHolidayCalendar

In [3]:
root = PurePath() 
raw_data = root / 'raw_data'

# files
economics_file = 'economics.csv'
yields_file = 'FED-SVENY.csv'


# import data
economics = pd.read_csv(economics_file)
yields = pd.read_csv(yields_file)


#========================================================================#
# clean data                                                             #
#========================================================================#

economics.index = pd.to_datetime(economics['sasdate'], format="%m/%d/%Y")
economics = economics.iloc[:,1:] # drop date column

# nan strategy is to drop as of now
economics = economics[~(economics.apply(np.isnan)).apply(any, axis=1)]
economics = economics.iloc[:-9,:] # done by inspection


yields.index = pd.to_datetime(yields['Date'], format="%Y-%m-%d")
yield_col = ['SVENY01',	'SVENY02', 'SVENY03', 'SVENY05', 'SVENY07', 
        'SVENY10', 'SVENY20', 'SVENY30']
yields = yields[yield_col]


#========================================================================#
# Join data by date                                                      #
#========================================================================#

# Right now we will move econ dates forward to next trading date

bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())

economics.index = economics.index +  bday_us

# 04/01/1999 gets moved when to 04/02/1999 it shouldn't
as_list = economics.index.tolist()
idx = as_list.index(dt.datetime(1999, 4, 2))
as_list[idx] = dt.datetime(1999, 4, 1)
economics.index = as_list

full = pd.concat([economics, yields], axis=1, join="inner")

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [4]:
display(full)

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,...,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx,SVENY01,SVENY02,SVENY03,SVENY05,SVENY07,SVENY10,SVENY20,SVENY30
1992-02-03,7968.831,6879.1,55.965,759384.431,164213.0,62.2167,70.4592,68.6771,80.0283,57.0098,90.1724,42.8645,53.6904,34.5759,99.6319,62.0864,74.9909,63.5608,78.8083,2804.0,0.296594,127207,117753,7.4,16.7,3277,2953,3182,1478,1704,442200,108311,22142,623.9,4612,16829,9990,6839,86169,22112,...,127.6989,1.7778,1.1825,122.3,120.8,113.8,98.6,18.995,109.8,138.600,131.000,124.500,185.600,127.800,117.300,149.900,138.900,135.600,135.900,66.557,134.522,65.342,58.315,11.89,13.67,11.30,68.8,2586.2,64066.00,189752.08,744.2595,17.0505,4.4364,5.1971,5.8127,6.7036,7.2638,7.7153,7.9983,8.1141
1992-03-02,7982.053,6884.7,56.067,762440.896,163721.0,62.7223,71.0547,69.2907,80.7498,58.1794,90.6269,43.3730,54.1051,34.9502,101.0182,62.7115,74.2890,63.8699,79.4206,2944.0,0.311205,127604,118144,7.4,17.1,3412,2801,3196,1425,1771,429500,108365,22127,622.5,4621,16805,9969,6836,86238,22113,...,132.8627,1.7238,1.1928,122.4,120.8,113.9,97.1,18.916,111.9,139.100,131.300,125.000,186.800,128.200,117.800,150.400,139.300,136.000,136.300,66.718,134.436,65.509,58.488,11.92,13.76,11.32,76.0,2611.3,62992.00,190887.95,749.7729,16.2227,4.6565,5.4234,6.0253,6.8573,7.3475,7.7146,8.0014,8.2955
1992-04-02,8011.806,6903.6,56.114,767521.889,164709.0,63.1981,71.5871,69.8247,81.5050,59.2725,91.1760,43.8082,54.5219,35.1690,101.1963,63.0556,77.4554,64.2653,79.6663,2840.0,0.301646,127841,118426,7.4,17.4,3296,2821,3130,1380,1750,418250,108519,22132,618.6,4603,16831,9977,6854,86387,22124,...,133.5395,1.7566,1.1874,122.5,120.9,114.1,98.1,20.243,112.7,139.400,130.700,125.600,187.900,128.300,118.100,150.900,139.700,136.400,136.600,66.899,134.701,65.535,58.709,11.96,13.77,11.36,77.2,2629.0,64280.00,190911.80,755.9014,16.1885,4.7037,5.5511,6.1791,6.9808,7.4074,7.7031,8.0214,8.3156
1992-05-04,8054.634,6941.7,56.404,760844.122,165612.0,63.4008,71.9885,70.2408,82.0383,61.1306,90.9797,44.1696,54.5327,35.5240,100.7038,63.4234,75.5640,63.7147,79.9329,2934.0,0.301108,128119,118375,7.6,17.8,3417,2788,3444,1460,1984,417400,108649,22135,615.6,4605,16835,9979,6856,86514,22109,...,130.7710,1.8095,1.1991,122.9,121.4,114.5,100.3,20.940,112.5,139.700,131.600,125.900,188.700,128.600,118.300,151.300,140.100,136.700,136.900,66.987,134.961,65.625,58.778,11.97,13.77,11.39,79.2,2652.9,62212.00,190975.14,765.6137,14.7280,4.3785,5.2845,5.9811,6.9180,7.4519,7.8425,8.1749,8.4609
1992-06-02,8088.220,6972.6,56.556,768864.805,166077.0,63.4358,71.8102,70.0255,81.6339,60.2972,90.8157,44.2850,54.7713,35.6103,101.7865,63.6275,72.5513,63.4713,79.9867,2898.0,0.288645,128459,118419,7.8,18.2,3518,2809,3758,1608,2150,418750,108715,22097,608.6,4584,16826,9966,6860,86618,22112,...,126.8355,1.8551,1.1960,123.4,121.9,115.1,101.6,22.375,113.9,140.100,132.100,126.400,189.600,129.100,118.500,151.700,140.600,137.100,137.300,67.107,134.667,65.925,58.863,12.01,13.88,11.41,80.4,2667.0,63021.00,192729.46,774.4532,14.7531,4.3904,5.1862,5.8100,6.6774,7.2027,7.6284,8.0645,8.3123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-04,17008.9

In [5]:
full.to_csv('combined_monthly_data.csv')